<a href="https://colab.research.google.com/github/baptonx7/enduraw-case-study/blob/main/GAP_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gpxpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.0 MB/s eta 0:00:00


In [18]:
import gpxpy
import pandas as pd
import numpy as np
import math

# 1) Load GPX geometry
with open("Lahti_run.gpx", "r") as f:   # replace with your GPX filename
    gpx = gpxpy.parse(f)

points = []
for track in gpx.tracks:
    for seg in track.segments:
        for p in seg.points:
            points.append({'lat': p.latitude, 'lon': p.longitude, 'ele': p.elevation})
df_gpx = pd.DataFrame(points)

# 2) Load split times from CSV
df_splits = pd.read_csv("Amelia Watkinson - Run-splits CSV.csv")

# Expecting columns like: "km","split_time"
# Make sure split_time is in seconds, otherwise convert
# Example if format is "mm:ss":
def time_to_sec(x):
    parts = str(x).split(":")
    if len(parts) == 2:
        return int(parts[0])*60 + int(parts[1])
    elif len(parts) == 3:
        return int(parts[0])*3600 + int(parts[1])*60 + int(parts[2])
    else:
        return float(x)

df_splits['time_s'] = df_splits['split_time'].apply(time_to_sec)

# 3) Compute cumulative distance from GPX
def haversine_m(lat1, lon1, lat2, lon2):
    R = 6371000.0
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2.0)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2.0)**2
    return R * 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

distances = [0.0]
for i in range(1, len(df_gpx)):
    d = haversine_m(df_gpx.loc[i-1,'lat'], df_gpx.loc[i-1,'lon'],
                    df_gpx.loc[i,'lat'], df_gpx.loc[i,'lon'])
    distances.append(d)
df_gpx['d_m'] = distances
df_gpx['cum_m'] = df_gpx['d_m'].cumsum()

# 4) Compute grade per km
km_bins = np.arange(1000, df_gpx['cum_m'].iloc[-1]+1000, 1000)
elev_changes, grades = [], []
for km in km_bins:
    mask = df_gpx['cum_m'] <= km
    elev = df_gpx.loc[mask, 'ele'].iloc[-1]
    if len(elev_changes)==0:
        elev_changes.append(0)
    else:
        elev_changes.append(elev - prev_elev)
    prev_elev = elev
grades = [ec/1000 for ec in elev_changes]  # grade = elev gain / horiz dist

# 5) Merge geometry + splits
df_splits['elev_change_m'] = elev_changes[:len(df_splits)]
df_splits['grade_frac'] = df_splits['elev_change_m']/1000.0
df_splits['pace_s_per_km'] = df_splits['time_s']  # since each split = 1 km

# 6) Apply simple GAP model
def gap_multiplier_simple(grade_percent, k_up=10.0, k_down=4.0):
    if grade_percent >= 0:
        return 1.0 + (k_up * grade_percent / 100.0)
    else:
        return max(0.7, 1.0 + (-k_down * abs(grade_percent) / 100.0))

df_splits['gap_pace_s_per_km'] = df_splits.apply(
    lambda r: r['pace_s_per_km'] / gap_multiplier_simple(r['grade_frac']*100),
    axis=1
)

# 7) Output results
df_splits['pace_min_per_km'] = df_splits['pace_s_per_km']/60
df_splits['gap_min_per_km'] = df_splits['gap_pace_s_per_km']/60

print(df_splits[['km','pace_min_per_km','gap_min_per_km','grade_frac']].head())

# Save to CSV
df_splits.to_csv("gpx_gap_splits.csv", index=False)


    km  pace_min_per_km  gap_min_per_km  grade_frac
0  0.4         1.783333        1.783333      0.0000
1  1.0         3.716667        2.845840      0.0306
2  1.2         5.116667        4.520024      0.0132
3  1.4         5.350000        5.397498     -0.0022
4  1.3         4.566667        5.161242     -0.0288


In [8]:

import gpxpy
import pandas as pd
import numpy as np
import math

gpx_file = "70.3WC Marbella Run Course.gpx"   # replace with your GPX path
target_gap_pace = 227          # target GAP in sec/km (e.g. 227 = 3:47/km)
segment_length = 500           # pacing plan resolution in meters

# 1) Parse GPX
with open(gpx_file, 'r') as f:
    gpx = gpxpy.parse(f)

distances = [0.0]
elevations = []

prev_point = None
for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            if prev_point is not None:
                d = point.distance_3d(prev_point)
                distances.append(distances[-1] + d)
            elevations.append(point.elevation)
            prev_point = point

# Make dataframe
df = pd.DataFrame({
    "distance": distances[:len(elevations)],
    "elevation": elevations
})

# 2) Smooth elevation (moving average)
df["elev_smooth"] = df["elevation"].rolling(window=5, center=True, min_periods=1).mean()

# 3) Compute slope
df["slope"] = df["elev_smooth"].diff().fillna(0) / df["distance"].diff().replace(0, np.nan).fillna(1)

# 4) GAP model
# Simple model: slope to pace multiplier
def gap_multiplier_simple(slope):
    # slope in %
    s = slope * 100
    if s > 10:
        s = 10
    if s < -10:
        s = -10
    return 1 + 0.03 * s  # +3% pace per +1% slope

# Convert slope to actual pace (sec/km)
df["pace"] = target_gap_pace * df["slope"].apply(gap_multiplier_simple)

# 5) Aggregate into 500m segments
segments = []
max_dist = df["distance"].iloc[-1]

for start in np.arange(0, max_dist, segment_length):
    end = min(start + segment_length, max_dist)
    mask = (df["distance"] >= start) & (df["distance"] < end)
    if mask.sum() > 0:
        avg_slope = df.loc[mask, "slope"].mean()
        avg_pace = df.loc[mask, "pace"].mean()
        segments.append({
            "start_m": int(start),
            "end_m": int(end),
            "avg_slope_%": round(avg_slope*100,2),
            "target_pace_sec_per_km": int(avg_pace),
            "target_pace_min_per_km": f"{int(avg_pace//60)}:{int(avg_pace%60):02d}"
        })

pace_plan = pd.DataFrame(segments)

# 6) Export to CSV
pace_plan.to_csv("pacing_plan.csv", index=False)
pace_plan.head()


,start_m,end_m,avg_slope_%,target_pace_sec_per_km,target_pace_min_per_km
0,0,500,-3.31,230,3:50
1,500,1000,-1.95,213,3:33
2,1000,1500,-0.37,224,3:44
3,1500,2000,-0.32,224,3:44
4,2000,2500,0.07,227,3:47
